In [1]:
import json
import torch
from PIL import Image
import torchvision.transforms as transforms
import numpy as np
import json
import requests
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(f'Using {device} for inference')


Using cpu for inference


In [2]:
import sys
print(sys.executable)

c:\Users\aswil\anaconda3\envs\glass_dots\python.exe


In [3]:
import argparse
import os
import torch 
import numpy as np
import torchvision
import torch.nn as nn
from torchvision.utils import save_image
import matplotlib.pyplot as plt
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch.utils.data.dataloader import DataLoader
from torch.nn.modules.utils import _pair
import torch.nn.functional as F
from statistics import mean
from torch.utils.tensorboard import SummaryWriter
from pathlib import Path
from PIL import Image
import gc
import h5py
import glob
from sklearn.model_selection import KFold
from sklearn.linear_model import Ridge
from sklearn.linear_model import LinearRegression
from scipy.stats.stats import pearsonr
from sklearn.decomposition import PCA

In [4]:
from torchvision.models import resnet50, ResNet50_Weights

# Using pretrained weights:
resnet = resnet50(weights="IMAGENET1K_V1")

resnet.eval().to(device)
image_type = {}

In [5]:
# for name, module in resnet.named_modules():
#     print(name)

In [6]:
def open_images(dir):
    ''' 
    (int, dict) -> NoneType
    
    Preprocess the data images and put them in a dictionary
    '''
    images = []
    
    for i in range (1, 501):
        #concentric
        #   image_type[i] = 0
        #
        #
        original_path = fr"C:\Users\aswil\Desktop\College\Internships\Bashivan_Lab\Projects\glass_dots\filler\{i}.jpg"
        image = original_path.replace("filler", dir)
        #Adjust for your system
        image_type[i] = 1
        with open(image, 'rb') as file:
            #print(image)


            img = Image.open(image).resize((224, 224))
    
            img_norm = np.array(img, dtype=np.float32) / 255.
        
            img_norm = (img_norm - [0.485, 0.456, 0.406]) / [0.229, 0.224, 0.225]
            
            images.append(np.transpose(img_norm, (2, 0, 1)))
        
             
    return images
    

In [7]:
#number_imgs = len(images_np)
print(image_type)


{}


In [8]:
def get_activation(name, activation):        
    def hook(model, input, output):
        activation[name] = output.detach()
    return hook

In [9]:
def get_activation_list(model, data_loader):
    all_activation = {}
    
    for i, data in enumerate(data_loader):
        _ = model(data.to(device))
        if len(all_activation) == 0: 
            for k in activation.keys(): 
                all_activation[k] = activation[k].cpu().numpy()
        else: 
            for k in activation.keys():
                all_activation[k] = np.concatenate([all_activation[k], activation[k].cpu().numpy()])
    
    return all_activation

In [10]:
activation = {}
def run(dir):
    images_np = np.array(open_images(dir), dtype=np.float32)
    dataloader = torch.utils.data.DataLoader(torch.from_numpy(images_np), batch_size=20, shuffle=False)
    model_layer_names = ["layer4.2", "avgpool"]
    x = 1

    for name, module in resnet.named_modules():
        if name in model_layer_names:
            #print(name)
            module.register_forward_hook(get_activation(name, activation))
    b = get_activation_list(resnet, dataloader)

    #print(b.keys())
    # print(all_activations["layer4.2"].shape)
    #print(b["avgpool"].shape)
    return b["avgpool"]
#a = run("starburst_3repeats_200dots")

In [11]:
# star = run("4r_200d_starburst")
# conc = run("4r_200d_concentric")
# hyper = run("4r_200d_hyperbolic")
# spiral = run("4r_200d_spiral")



In [12]:
# output = np.concatenate((star, conc, hyper, spiral), axis=0)
# # output = np.concatenate((star, conc), axis=0)
# print(output.shape)

In [14]:
labels = np.array([])
a = [0] * 500
b = [1] * 500
c = [2] * 500
d = [3] * 500
for i in range (4):
    for j in range (500):
        break

labels = np.append([a, b], [c, d])
print(labels)
print(len(labels))

[0 0 0 ... 3 3 3]
2000


In [ ]:
# spiral = run("3r_200d_spiral")
# conc = run("3r_200d_concentric")
# test_output = np.concatenate((spiral, conc), axis=0)
# test_labels = np.array([])
# labels = np.array([])
# test_labels = np.append(a, b)


KeyboardInterrupt: 

In [15]:
# print(test_labels)
print(labels)

[0 0 0 ... 3 3 3]


In [ ]:
# output = np.squeeze(output)
# test_output = np.squeeze(test_output)
# print(len(output))
# print(len(labels))
# print(labels)

2000
2000
[0 0 0 ... 3 3 3]


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn import metrics


X_train, X_test, y_train, y_test = train_test_split(output, labels, test_size=0.99,random_state=109)
print(len(X_train))
print(len(X_test))
print(len(y_train))
print(len(y_test))

clf = svm.SVC(kernel='linear') # Linear Kernel

clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)

print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

print("Precision:",metrics.precision_score(y_test, y_pred))

print("Recall:",metrics.recall_score(y_test, y_pred))


NameError: name 'output' is not defined

In [16]:
import numpy as np 
from sklearn.datasets import load_iris
from sklearn.model_selection import KFold
from sklearn.svm import SVC
# print(output.shape)
# Y = labels
# print(labels.shape)
def kfold(output, labels):
    N_FOLDS = 5

    kf = KFold(n_splits=N_FOLDS,shuffle=True)
    clf = SVC(kernel='linear')
    preds = np.zeros_like(labels)
    for train_index, test_index in kf.split(output):
        clf.fit(output[train_index], labels[train_index])
        preds[test_index] = clf.predict(output[test_index])
    print(f"Accuracy: {100*np.sum(preds == labels)/preds.size:.2f}%")
    return(100*np.sum(preds == labels)/preds.size)
# kfold(output, labels)

In [18]:
results = []
for i in range(3,5):
    num = i
    # path = str(num) + "r_200d_"
    starburst = run(str(num) + "r_50d_starburst")
    concentric = run(str(num) + "r_50d_concentric")
    hyperbolic = run(str(num) + "r_50d_hyperbolic")
    spiral = run(str(num) + "r_50d_spiral")
    output = np.squeeze(np.concatenate((starburst, concentric, hyperbolic, spiral), axis=0))
    print(output.shape)
    ans = kfold(output, labels)
    results.append(ans)
print(results) 
    
    

(2000, 2048)
Accuracy: 98.50%
(2000, 2048)
Accuracy: 99.15%
[98.5, 99.15]


In [19]:
fade_results = []
for i in range(3):
    if i == 0:
        fade = "lightfade"
    elif i == 1:
        fade = "midfade"
    else:
        fade = "heavyfade"
    # path = str(num) + "r_200d_"
    starburst = run("3r_50d_starburst_"+ fade)
    concentric = run("3r_50d_concentric_" + fade)
    hyperbolic = run("3r_50d_hyperbolic_" + fade)
    spiral = run("3r_50d_spiral_" + fade)
    output = np.squeeze(np.concatenate((starburst, concentric, hyperbolic, spiral), axis=0))
    print(output.shape)
    ans = kfold(output, labels)
    fade_results.append(ans)
print(fade_results)

(2000, 2048)
Accuracy: 98.55%
(2000, 2048)
Accuracy: 98.50%
(2000, 2048)
Accuracy: 98.00%
[98.55, 98.5, 98.0]


In [17]:
distraction_results = []
for i in range(3):
    if i == 0:
        dist = "50dist"
    elif i == 1:
        dist = "100dist"
    else:
        dist = "200dist"
    # path = str(num) + "r_200d_"
    starburst = run("3r_50d_starburst_"+ dist)
    concentric = run("3r_50d_concentric_" + dist)
    hyperbolic = run("3r_50d_hyperbolic_" + dist)
    spiral = run("3r_50d_spiral_" + dist)
    output = np.squeeze(np.concatenate((starburst, concentric, hyperbolic, spiral), axis=0))
    print(output.shape)
    ans = kfold(output, labels)
    distraction_results.append(ans)
print(distraction_results)

(2000, 2048)
Accuracy: 97.30%
(2000, 2048)
Accuracy: 98.15%
(2000, 2048)
Accuracy: 97.55%
[97.3, 98.15, 97.55]


In [18]:
distraction_res2 = []
# for i in range(3):
#     if i == 0:
#         dist = "50dist"
#     elif i == 1:
#         dist = "100dist"
#     else:
dist = "200dist"
# path = str(num) + "r_200d_"
starburst = run("2r_50d_starburst_"+ dist)
concentric = run("2r_50d_concentric_" + dist)
hyperbolic = run("2r_50d_hyperbolic_" + dist)
spiral = run("2r_50d_spiral_" + dist)
output = np.squeeze(np.concatenate((starburst, concentric, hyperbolic, spiral), axis=0))
print(output.shape)
ans = kfold(output, labels)
distraction_results.append(ans)
print(distraction_res2)

In [58]:
a = run("starburst_3repeats_200dots")
b = run("starburst_5repeats_200dots")
c = np.concatenate((a, b), axis=1)
print(c.shape)
print(a[1][0])
print(b[1][0])
# print("END")
# print(b)
# print("END")
print(c[1][0])
print(c[1][2048])
# print(a)
# np.reshape(b, (20, -1))
# print (b.shape)

layer4.2
avgpool
dict_keys(['layer4.2', 'avgpool'])
(100, 2048, 1, 1)
layer4.2
avgpool
dict_keys(['layer4.2', 'avgpool'])
(100, 2048, 1, 1)
(100, 4096, 1, 1)
[[0.01813324]]
[[0.02748377]]
[[0.01813324]]
[[0.02748377]]


In [83]:
a = run("starburst_3repeats_200dots")
b = run("starburst_5repeats_200dots")
a.reshape(100, 100352)
print(a.shape)

(100, 2048, 7, 7)


In [12]:
for layer in ["layer1.0", "layer1.1", "layer1.2", "layer2.0", "layer2.1", "layer2.2", "layer2.3", "layer3.0", "layer3.1", "layer3.2", "layer3.3", "layer3.4", "layer3.5", "layer4.0", "layer4.1", "layer4.2"]:
    act_now = all_activations[layer]

In [13]:
seed=10
PCA_activations={}
for layer in ["layers.0.0", "layers.0.1", "layers.0.2", "layers.1.0", "layers.1.1", "layers.1.2", "layers.1.3", "layers.2.0", "layers.2.1", "layers.2.2", "layers.2.3", "layers.2.4", "layers.2.5", "layers.3.0", "layers.3.1", "layers.3.2"]:
    act_now = all_activations[layer]
    pca=PCA(random_state=seed)
    pcaact_now = pca.fit_transform(act_now.reshape(act_now.shape[0],-1))
    PCA_activations[layer] = pcaact_now
print(PCA_activations.keys())

KeyError: 'layers.0.0'

In [ ]:
#for i, (comp, var) in enumerate(zip(pca.components_, pca.explained_variance_)):
#    comp = comp * var  # scale component by its variance explanation power
#    plt.plot(
#        [0, comp[0]],
#        [0, comp[1]],
 #       label=f"Component {i}",
  #      linewidth=2,
   #     color=f"C{i + 2}",
    #)
#plt.gca().set(
#    aspect="equal",
#    title="2-dimensional dataset with principal components",
#    xlabel="first feature",
#    ylabel="second feature",
#)
#plt.legend()
#plt.show()

In [ ]:
print(pca.singular_values_)

[1.5198561e+02 1.3453784e+02 1.2953775e+02 1.1676004e+02 1.1431002e+02
 4.1520652e-05]


In [ ]:
pca2 = PCA(random_state=seed)
graph = pca2.fit(PCA_activations['layers.0.0'])
print(graph)

PCA(random_state=10)


In [ ]:
print(PCA_activations)

{'layers.0.0': array([[-2.1707344e+01, -1.2113386e+02, -3.2977287e+01,  1.6851707e+02,
        -2.6765476e+01,  1.0878677e-02],
       [-1.2197134e+02, -1.1759546e+01, -1.4020808e+02, -1.0499556e+02,
        -1.9099089e+01,  1.0878062e-02],
       [-7.2329514e+01, -5.7997044e+01,  1.6194826e+02, -5.5723648e+01,
         8.7933571e+01,  1.0877628e-02],
       [-4.5620720e+01,  1.9023410e+02,  3.9837315e+01,  7.0893242e+01,
        -4.2835846e+01,  1.0877129e-02],
       [ 1.3570551e+02,  3.4418690e+01, -7.0334824e+01, -2.8729482e+00,
         1.4388763e+02,  1.0875755e-02],
       [ 1.2597929e+02, -3.3742943e+01,  4.1742306e+01, -7.5844055e+01,
        -1.4312596e+02,  1.0875243e-02]], dtype=float32), 'layers.0.1': array([[-9.1298241e+01, -1.2664704e+02,  1.5656744e+01,  1.9698540e+02,
        -1.4403738e+02,  2.7895514e-02],
       [ 1.2955975e+02, -9.4451500e+01,  2.0136656e+02, -1.3450052e+02,
        -4.4117939e+01,  2.7893543e-02],
       [ 3.2490219e+01, -1.6514983e+02, -1.4977782

In [ ]:
import numpy as np 
from sklearn.datasets import load_iris
from sklearn.model_selection import KFold
from sklearn.svm import SVC

npa = np.array

In [85]:
data = load_iris()
X = data.data
Y = data.target
print(type(data.target))
print(X.shape, Y.shape)
print(Y)

<class 'numpy.ndarray'>
(150, 4) (150,)
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2]


In [82]:
N_FOLDS = 5

kf = KFold(n_splits=N_FOLDS,shuffle=True)
clf = SVC(kernel='linear')
preds = np.zeros_like(Y)
for train_index, test_index in kf.split(X):
    clf.fit(X[train_index], Y[train_index])
    preds[test_index] = clf.predict(X[test_index])
print(f"Accuracy: {100*np.sum(preds == Y)/preds.size:.2f}%")

Accuracy: 97.33%


In [ ]:
print(f"Predictions: {preds}")
print(f"Targets: {Y}")

Predictions: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 1 1 1
 1 1 1 1 1 1 1 1 1 2 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 1 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 1 2 2 2 2 2 2 2 2 2
 2 2]
Targets: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2]
